<center>
<h1 style="color:red; font-size:70px;"> Esercitazione 6</h1>

In [2]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris

<h1 style="color: blue; font-size:50px;"> Esercizio 1 - Early stopping</h1>

In [8]:
# La prima fase di questo esercizio consiste nell'importare il dataset IRIS, che
# restitusce un dizionario le cui flag sono:
#       - 'data': matrice con lughezze sepali e petali
#       - 'target': etichetta del fiore corrispondente
#       - 'feature_names': elenco di stringhe per nomi dati
#       - 'target_names': elenco di stringhe per nomi fiori
# In particolare 'data' è un array bidimensionale di forma (150, 4) in cui ogni riga
# rappresenta un fiore, e ogni colonna rappresenta una delle caratteristiche dello stesso. 
# Per ogni fiore sono quindi prese in considerazione 4 features, ossia lunghezza e larghezza
# del sepalo e del petalo.
iris = load_iris()
# Per convertire quanto ottenuto come output della funzione load_iris() in un oggetto di pandas 
# (che è comodo per gestiri tali tipologie di dataset) è possibile specificare nel costruttore 
# del dataFrame i dati contenuti in iris (ossia lunghezza e larghezza di sepali e petali di ciascun 
# fiore) ed i nomi delle colonne.
irisDf = pd.DataFrame(data = iris.data, columns = iris.feature_names)
# In un secondo momento vanno aggiunte delle altre colonne al dataframe, ossia l'etichetta numerica 
# target, che indica la classe di appartenenza del fiore, ed il nome della classe in questione.
# Questa operazione non può essere effettuata in fase di inizializzazione.
irisDf['target'] = iris.target
irisDf['target_name'] = irisDf['target'].apply(lambda i: iris.target_names[i])

# Stampo a video il contenuto del dataframe
print("Contenuto del dataframe")
irisDf

Contenuto del dataframe


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,virginica
146,6.3,2.5,5.0,1.9,2,virginica
147,6.5,3.0,5.2,2.0,2,virginica
148,6.2,3.4,5.4,2.3,2,virginica
